In [1]:
!pip install transformers datasets faiss-cpu sentence-transformers

In [2]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import torch

In [7]:
from transformers import GPTNeoForCausalLM, GPT2Tokenizer

# Load a smaller GPT-Neo model
model_name = "EleutherAI/gpt-neo-125M"
model = GPTNeoForCausalLM.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# Set the device to CPU or GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
model.eval()

config.json:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/526M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/357 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

GPTNeoForCausalLM(
  (transformer): GPTNeoModel(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(2048, 768)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPTNeoBlock(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPTNeoAttention(
          (attention): GPTNeoSelfAttention(
            (attn_dropout): Dropout(p=0.0, inplace=False)
            (resid_dropout): Dropout(p=0.0, inplace=False)
            (k_proj): Linear(in_features=768, out_features=768, bias=False)
            (v_proj): Linear(in_features=768, out_features=768, bias=False)
            (q_proj): Linear(in_features=768, out_features=768, bias=False)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPTNeoMLP(
          (c_fc): Linear(in_features=768, out_features=3072, bias=True)
          (c_proj): Linear(in_fe

In [9]:
# Load a Sentence Transformer model to convert sentences to embeddings
embedder = SentenceTransformer('all-MiniLM-L6-v2')

# Dummy corpus of documents for retrieval
documents = [
    "The Eiffel Tower is located in Paris.",
    "The Great Wall of China is visible from space."
    # "Artificial Intelligence is transforming the world.",
    # "The Grand Canyon is one of the seven wonders of the world."
]

# Convert the documents into embeddings
document_embeddings = embedder.encode(documents)

# Initialize FAISS index
dimension = document_embeddings.shape[1]  # Dimension of the embeddings
faiss_index = faiss.IndexFlatL2(dimension)  # L2 distance index
faiss_index.add(np.array(document_embeddings))  # Add document embeddings to index


In [17]:
def generate_without_rag(query):
    """Generate a response from the LLM without any external document retrieval."""
    # Tokenize the query and generate response
    inputs = tokenizer(query, return_tensors="pt").to(device)

    # Generate output without any retrieved document
    outputs = model.generate(
        inputs['input_ids'],
        attention_mask=inputs['attention_mask'],
        max_length=50,
        num_return_sequences=1,
        pad_token_id=tokenizer.eos_token_id
    )

    # Decode and return the generated response
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_text


def generate_with_rag(query):
    """Generate a response from the LLM with document retrieval using FAISS."""
    # Retrieve the relevant document using FAISS
    query_embedding = embedder.encode([query])
    _, indices = faiss_index.search(np.array(query_embedding), k=1)
    relevant_document = documents[indices[0][0]]
    
    # Augment the query with the retrieved document
    augmented_prompt = f"Using the information from the document, answer the question.\n\nDocument: {relevant_document}\n\nQuestion: {query}\n\nAnswer:"
    
    # Tokenize the augmented prompt and generate response
    inputs = tokenizer(augmented_prompt, return_tensors="pt").to(device)

    outputs = model.generate(
        inputs['input_ids'],
        attention_mask=inputs['attention_mask'],
        max_length=50,
        num_return_sequences=1,
        pad_token_id=tokenizer.eos_token_id,
        no_repeat_ngram_size=3  # This prevents repeating the same n-grams
    )

    # Decode and return the generated response
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_text

# Example usage
query = "Where is the Eiffel Tower located?"

# Generate response without RAG
response_without_rag = generate_without_rag(query)
print("Response WITHOUT RAG:")
print(response_without_rag)

# Generate response with RAG
response_with_rag = generate_with_rag(query)
print("\nResponse WITH RAG:")
print(response_with_rag)


Response WITHOUT RAG:
Where is the Eiffel Tower located?

The Eiffel Tower is located in the city of Berlin, Germany. The tower is located in the city of Berlin, Germany. The tower is a part of the city of Berlin, Germany

Response WITH RAG:
Using the information from the document, answer the question.

Document: The Eiffel Tower is located in Paris.

Question: Where is the Eiffel Tower located?

Answer: The Tower is in Paris, France.
